# 8. Deploy IoT Edge Modules

In this section, we will deploy LVA & Inference Server modules (IoT Edge Runtime specific Docker container images) to an IoT Edge device.

## 8.1. Setup deployment manifest file
First, we will write a manifest file in Json format and specify the details of the modules (IoT Edge Runtime specific docker container images) that will be deployed into the edge device. In this step, we will use the [deployment.01.template.json](deployment.01.template.json) manifest file for module deployment. Familiarize yourself with the content of the [deployment.01.template.json](/utilities/video-analysis/jupyter/deployment.01.template.json) manifest file. The sample manifest is organized similarly to the following image:  

<img src="documents/_deployment_modules_0.png" width=300px/>  

As shown in the above image's enumeration, section 2 of the Json file defines the system modules to be deployed onto the edge device. These two system modules (edgeAgent, edgeHub) are necessary modules in each IoT Edge device. In section 3 of the Json file, we define the custom modules that we want to deploy to the edge device.  

The three custom modules in section 3 of the Json file are 
* lvaEdge - node where we define the lvaEdge-specific details, such as from which Container Registry address it should be downloaded. (You may use other reference names, but the name must be consistent in other locations within the manifest file or in any other related code.)
* isServer - the reference name we gave for our inference server module that we developed and uploaded into ACR.
* rtspsim - the reference name given to another container image in ACR. This module simulates an IP camera by playing a user-defined video file as if it were being streamed from an IP camera. This module is optional and placed here to aid users who don't have an IP camera but still want to test LVA.

For more details about IoT Edge modules, you may refer to [Azure IoT Edge documentation](https://docs.microsoft.com/en-us/azure/iot-edge/about-iot-edge).

If you check the contents of the deployment manifest file, you will see placeholders for some values. These are:

| **Placeholder** | **More Information** |
|:-----------------------------------|:---------------------------------------------------------------------------------------------------------------------------------|
| SUBSCRIPTION_ID | The ID associated with your Azure subscription, typically containing dashes within the string.<br>A subscription represents a grouping of Azure resources. An invoice is generated at the subscription scope. | 
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| RESOURCE_GROUP | The name of the resource group that holds your resources.<br>A resource group is a container that holds related resources for an Azure solution. | 
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| AMS_ACCOUNT | The name of your Azure Media Services (AMS) resource.<br>AMS is a cloud-based media workflow platform to index, package, protect, and stream video. |
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| AAD_TENANT_ID | The Azure Active Directory (AAD) tenant ID, which is used as a reference to a directory.<br>An AAD directory object holds security-related objects such as user accounts, applications, and groups.| 
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| AAD_SERVICE_PRINCIPAL_ID | The ID associated with the AAD's service principal.<br>A service principal is a security identity used by user-created apps, services, and tools to access Azure resources. | 
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| AAD_SERVICE_PRINCIPAL_SECRET | A secret string associated with the AAD's service principal.<br>The AAD service principal secret adds a layer of security at the service principal scope.|
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| INPUT_VIDEO_FOLDER_ON_DEVICE | The path pointing to the folder in which the desired input video is stored.<br>This path must be specified in order for LVA to find the correct videos to process.| 
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| OUTPUT_VIDEO_FOLDER_ON_DEVICE | The path pointing to the folder in which the processed video output will be stored.<br>After processing is complete, the processed video will be sent and stored in this folder. | 
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| CONTAINER_REGISTRY_USERNAME_myacr | The username that Azure Container Registry (ACR) will use to authenticate with the registry.<br>ACR is a registry of Docker and Open Container Initiative (OCI) images.| 
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| CONTAINER_REGISTRY_PASSWORD_myacr | The password that Azure Container Registry (ACR) will use to authenticate with the registry.<br>When run, a Active Directory token is created to authenticate your session with the container registry.| 
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|
| CONTAINER_IMAGE_NAME | The name of the container image that will be used for deployment.<br>Once uploaded to ACR, it can be called through CONTAINER_REGISTRY_USERNAME_myacr.azurecr.io/CONTAINER_IMAGE_NAME. |
|-----------------------------------|---------------------------------------------------------------------------------------------------------------------------------|

In the previous tutorial, we pushed our inference server container image to the cloud using Azure Container Registry (ACR). In doing so, ACR created a .env file containing the credentials to your container registry. These credentials are shared with your IoT Edge device so that it has access to pull the container image.

We will use the credentials found in the .env file to create an IoT Edge device module deployment manifest file, which is written in Json format. The credentials found in the .env file will replace the placeholders we set in our template [deployment.01.template.json](deployment.01.template.json) manifest file.

**How does this work?** The deployment manifest file is a file that defines exactly which modules you want deployed on a device, how they should be configured, and how they can communicate with each other and the cloud. When you transform the template into a true deployment manifest, the pointers are replaced with values taken from other solution files. Locate the two common placeholders in your deployment template:

In the registry credentials section, the address is autofilled from the information you provided when you created the solution. However, the username and password reference the variables stored in the .env file. This configuration is for security, as the .env file is git ignored, but the deployment template is not.
In the SampleModule section, the container image isn't filled in even though you provided the image repository when you created the solution. This placeholder points to the module.json file inside the SampleModule folder. If you go to that file, you'll see that the image field does contain the repository, but also a tag value that is made up of the version and the platform of the container. You can iterate the version manually as part of your development cycle, and you select the container platform using a switcher that we introduce later in this section.
Provide your registry credentials to the IoT Edge agent
The environment file stores the credentials for your container registry and shares them with the IoT Edge runtime. The runtime needs these credentials to pull your container images onto the IoT Edge device.

The IoT Edge extension tries to pull your container registry credentials from Azure and populate them in the environment file. Check to see if your credentials are already included. If not, add them now:

Open the .env file in your module solution.
Add the username and password values that you copied from your Azure container registry.
Save your changes to the .env file.

To learn more about 

<JasonX> below there is content on how to deploy modules into edge device with screenshots. Please organize them accrdingly to have section 8 completed. at the end of section 8, user must know how to deploy edge modules into an edge device. Below there is additional content on how to deploy media graph, remove them. We will copy paste it into section 9 (like secction 8) to show how to deploy media graph...


```
.










.



























.
```

In [ ]:
## 8.1. Get global variables
We will read the previously stored variables

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

acrUserName = get_key(envPath, "acrUserName")
acrPassword = get_key(envPath, "acrPassword")
containerImageName = get_key(envPath, "containerImageName")
azureSubsctiptionId = get_key(envPath, "azureSubsctiptionId")
resourceGroupName = get_key(envPath, "resourceGroupName")
mediaServiceName = get_key(envPath, "mediaServiceName")
aadTenantId = get_key(envPath, "aadTenantId")
aadServicePrincipalId = get_key(envPath, "aadServicePrincipalId")
aadServicePrincipalSecret = get_key(envPath, "aadServicePrincipalSecret")
iotDeviceId = get_key(envPath, "iotDeviceId")
iotHubConnString = get_key(envPath, "iotHubConnString")

## 8.2. Update .Env file for module deployment manifests

We will use an IoT Edge Device Module Deployment Manifest file (deployment.lvaedge_aixserver_rtspsim.template.json) which has parameter placeholders that we will replace with our service names, credentials etc. that we created in the previous sections. This placeholder parameters are auto set by VSCode using the **".env"** file. So below cell will update the .env file with the required parameters.

In [ ]:
set_key(envPath, "CONTAINER_REGISTRY_USERNAME_myacr", acrUserName)
set_key(envPath, "CONTAINER_REGISTRY_PASSWORD_myacr", acrPassword)
set_key(envPath, "CONTAINER_IMAGE_NAME", containerImageName)
set_key(envPath, "SUBSCRIPTION_ID", azureSubsctiptionId)
set_key(envPath, "RESOURCE_GROUP", resourceGroupName)
set_key(envPath, "AMS_ACCOUNT", mediaServiceName)
set_key(envPath, "AAD_TENANT_ID", aadTenantId)
set_key(envPath, "AAD_SERVICE_PRINCIPAL_ID", aadServicePrincipalId)
set_key(envPath, "AAD_SERVICE_PRINCIPAL_SECRET", aadServicePrincipalSecret)
set_key(envPath, "INPUT_VIDEO_FOLDER_ON_DEVICE", "/lvafolders/input")
set_key(envPath, "OUTPUT_VIDEO_FOLDER_ON_DEVICE", "/lvafolders/output")

## 8.3. Setup Local Folders on the IoT Edge Device
Some samples, deployment manifests may not need these additional folders we are creating in this section but as place holder we are creating for all deployments.

Connect to the Iot Edge device with a terminal window session and create the below two folders on the "IoT Edge Device".

Using terminal window, create two folders at any location in your edge device. We prefered folowing two folder names with full path:  
```
    /lvafolders/input  
    /lvafolders/output  
```

You can give any name to these folders and create them under any desired root folder. But full path values that you prefered must be same in the other parts of this section. Both folder must have read/write access for all users. Below are the commands that we use to create them and set their access permissions. Run the below commands **on the IoT Edge Device** through the terminal window.

```
sudo mkdir -p /lvafolders/input
sudo mkdir /lvafolders/output
```

Finally we set these folder's access permissions.  
```
sudo chmod 777 -R /lvafolders
```


## 8.4. Reset the IoT Edge Device and Deploy LVA & Inference Server Modules

We will reset the IoT Edge Device by uploading a deployment manifest that consist of only two system module (IoTHub, IoTEdge)

Follow these steps:

1. Right click on the "deployment.reset.json" file  
<img src="documents/_deployment_reset.png" width=500px/>  

2. Click on "Create Deployment for Single Device"  
    a. If it is the first time using current IoT Hub service that was created in the previous sections, VSCode will ask you to enter "IoT Hub Connection String", you can use the value of "iotHubConnString" key, which can be located in the ".env" file.  
    b. "Create Deployment for Single Device" command will open a window on top edge of the VSCode and will ask you the IoT Edge Device name that you want to make this deployment. Since you entered the IoT Hub name in step (a), it will automatically find the device names and list for you. For this sample we have only a single device which we name it with the value of variable "iotDeviceId" in the second section where we defined the variables. Select this device in the drop down list.  
<img src="documents/_select_edge_device.png" width=500px/>  

3. Now right click on the file named "deployment.lvaedge_aixserver_rtspsim.template.json". In the pop-up menu, click on the "Create IoT Edge Deployment Manifest". In this step, VS code will auto read the contents of ".env" file and uses the values of the variables to write over some place holder variables inside the "deployment.lvaedge_aixserver_rtspsim.template.json". After these replacement, new version of the file with the name "deployment.lvaedge_aixserver_rtspsim.amd64.json" will be placed under the newly created "config" folder in the same working directory.  
<img src="documents/_deployment_modules_1.png" width=600px/>  

4. Right click on the file "/config/deployment.lvaedge_aixserver_rtspsim.amd64.json", In the pop-up menu, click on the "Create Deployment for Single Device"  
<img src="documents/_deployment_modules_2.png" width=600px/>  

5. Like in step 2.b, VSCode will ask the device name to where the deployment will be made. Here again select the device name with the value of variable "iotDeviceId" in the second section where we defined the variables.

6. Depending on your Internet speed, in seconds to minutes, modules will be pulled from the cloud and deployed into your edge device. In the VSCode, under "AZURE IOT HUB" panel, you can see the modules that are uploaded on to your edge device by refreshing.  
<img src="documents/_iotedgedevice_view.png" width=400px/>  

As in the above screen shot you must see "running" word near each module name (see region 3 in the above screenshot)  

Or you can run the following command in a terminal window on Iot Edge Device:  
```
sudo iotedge list
```

and the result of the above command will look like something similar to below with all module status are "running":  
```
mksa@mknuc02:~/Desktop/aiext$ sudo iotedge list
[sudo] password for mksa: 
NAME             STATUS           DESCRIPTION      CONFIG
aixServer        running          Up 6 minutes     mkregistry:55000/aix:latest
edgeAgent        running          Up 7 minutes     mcr.microsoft.com/azureiotedge-agent:1.0
edgeHub          running          Up 6 minutes     mcr.microsoft.com/azureiotedge-hub:1.0
lvaEdge          running          Up 6 minutes     lvacontainers.azurecr.io/lvaedge:1.0.01227.756
rtspsim          running          Up 6 minutes     mcr.microsoft.com/lva-utilities/rtspsim-live555:1.2
```

At this point we succesfully deployed  modules into Iot Edge Device.  



## 7.5. Deploy / Manage Media Graph on LVA edge module
At this point of the sample, we should have all required modules deployed to edge device, up and running.  
<img src="documents/_architecture.jpg" width=500px/>  

Now we will deploy Media graph into the "lvaEdge" module. Here "lvaEdge" is the name that we assign to the LVA module inside the module deployment template json file. Media graph is also a json file that defines a media flow pipeline from media sources to the target sinks and any anytics processes inbetween. Organization of the media graph file consists of following modules:  
```
{
    "@apiVersion": "1.0",

    "name": "SampleMediaGraphOrganization",

    "properties":{

        "parameters": ...

        "sources": ...

        "processes": ...

        "sinks": ...
    }
}
```

In the above media graph json file, apart from sources, processors and sinks, parameters section is the critical part of the media graph that defines all required parameters to run the media graph. In this section we define the addresses of the media sources, their access credentials and parameters related to the process modules like fps rate, sensitivity of motion detection module etc. Each media graph may have different media flow i.e. one may have motion detection processor while the other may not. User may modify this media graph according to problem that they want to solve in their scenarios.  



### 7.5.1. Sample Media Graph
In this sample project we provide a sample media graph file "GraphTopology_HTTPExtension_Jpeg_01.json".  Media graph in this file ingests a video stream from a single IP camera, runs motion detector processor on the stream and if there is a motion, it sends the stream to two different target: first one is the Azure Media Services on the cloud to archieve the stream's motion chunks. Other target is the custom AI module that we developed in this sample. AI module analyses the stream (a chunk with motion) and than the result (json format inference result) is sent to IoT Hub.

### 7.5.2. Setting Parameters in Media Graph
There are two ways to set the parameters in the media graph:  
1. Update the default values of the parameters in "GraphTopology_HTTPExtension_Jpeg_01.json", which is not the prefered way!  
2. Note the parameter names and data types in "GraphTopology_HTTPExtension_Jpeg_01.json", and override their values in the deployment command that we will show in the next section.
  

### 7.5.3. Deploy the Media Graph with Custom parameters
Below we will explain the following operations:
1. Setting (deploying) Media Graph Topology on the IoT Edge Device's lvaEdge module
2. Creating a Media Topolgy Instance from the one set in step 1
3. Activate the Topology instance that is set in step 2

Also we will show:
- How to list and delete the already set Graph Topologies on the Device
- How to list and delete the already set Graph Instances on the Device
- How to acticate / deactivate the Instances

#### 7.5.3.1. Install prerequisites
We will use a simple python script to deploy our Media Graph json into the "lvaEdge" module. Which is a process of sending a file into IoT Edge Device over Internet through IoT Edge Hub service. To do so, we need to install following Microsoft developed generic Python packages that helps for interaction with IoT Hub Services.

!Important: As mentioned in the first section of this tutorial, you must use the right pip command (pip or pip3) depending on your Python installation. This sample is tested with > Python3.6 version and here we use pip3 command to install the packages into our python3 environment.

In [ ]:
!pip3 install azure-iot-device
!pip3 install azure-iot-hub

#### 7.5.3.2. Import Helper Python class to deploy Media Graphs
Following line will import the custom Python class that we wrote to help us deploying Media edge graphs. After import, we will instantiate a graph manager object with IoT Hub, IoT Edge Device and Graph API version details.

In [ ]:
from graph_manager import GraphManager

moduleId = "lvaEdge" # Must be same as the name that we assigned to LVA module in the "deployment.lvaedge_aixserver_rtspsim.template.json" file
operationsApiVersion = "1.0"  # Must be same as the version number in the "GraphTopology_HTTPExtension_Jpeg_01.json" file

graphManager = GraphManager(iotHubConnString, iotDeviceId, moduleId, operationsApiVersion)

#### 7.5.3.3. List existing Graph Topologies and Instances


In [ ]:
# List topologies
response = graphManager.GenericCall("GraphTopologyList", {})
print(response)

In [ ]:
# List instances
response = graphManager.GenericCall("GraphInstanceList", {})
print(response)

#### 7.5.3.4. Set Topology
Below code will deploy "GraphTopology_HTTPExtension_Jpeg_01.json" file into the lvaEdge module running on our IoT Edge Device.

In [ ]:
# You can define a topologyFile or a topologyUrl that you want to deploy into the module. Here we point to our sample Media Graph Topology File.
operationParams = {
                    "topologyFile": "GraphTopology_HTTPExtension_Jpeg_01.json"
                    }

In [ ]:
# Set Graph Topology
response = graphManager.GraphTopologySet(operationParams)
print(response)

#### 7.5.3.5. Set Topology Instance

Below code will set the values of the parameters mentined in "GraphTopology_HTTPExtension_Jpeg_01.json" and will create a Topology Instance. If you modify the topology, add new parameters etc. you should update the below parameter list accordigly.  

! Important:  
- Be sure that the value of "topologyName" parameter below and the same parameter in the "GraphTopology_HTTPExtension_Jpeg_01.json" file to be the same.  
- Since we dont have physical IP camera set for this sample, we use a virtual IP camera simulator. In the above steps, with module deployment manifest, we already deployed a simulator module which plays "sample.mkv" video file as it played from an IP camera. So in the below parameter list, the parameter "rtspUrl" points to the URL address of this IoT Module (rtsp://rtspsim:554) with the full path of the video file that we want to play. You can replace this parameter's value with full RTSP address of your phyical IP camera.  
- rtspUsername and rtspPassword are dummy values for our simulator because it doesnt require any authentication. But if your source stream (i.e. phyical IP Camera) requires authentication, than put appropriate values for these parameters.  
- Value of "motionSensitivity" parameter can be one of: {low, medium, high}. As the name implies, it sets the sensitivity of the motion detection processor.
- Value of "grpcAIServerAddress" parameter is pointing to the address of the inference server. Name of the module that we developed in this sample and deployed in our module deployment manifest was set to "aixServer" which was listening inference request on "/score" endpoint through port 44001. So here we set exact address of the scoring endpoint.
- "hubSinkOutputName" parameter sets the IoT Hub channel name which ingest inference results into IoTHub

In [ ]:
mediaGraphTopologyParameters = {
          "name": "Sample-Graph-1",
          "properties": {
            "topologyName": "SampleTopology",
            "description": "Sample graph description",
            "parameters": [
              {
                "name": "rtspUrl",
                "value": "rtsp://rtspsim:554/media/sample.mkv"
              },
              {
                "name": "rtspUsername",
                "value": "username"
              },
              {
                "name": "rtspPassword",
                "value": "password"
              },
              {
                "name": "motionSensitivity",
                "value": "medium"
              },
              {
                "name": "httpAIServerAddress",
                "value": "http://aixServer:5001/score"
              },
              {
                "name": "hubSinkOutputName",
                "value": "inferences"
              }
            ]
          }
        }

After setting above parameters as in Python dictionary data structure, now we can set an instance of previously deployed topology on the edge device with the above custom parameters.

In [ ]:
# Set topology instance
graphManager.GenericCall("GraphInstanceSet", mediaGraphTopologyParameters)
print(response)

#### 7.5.3.6. Activate Topology Instance
Activates the Topology instance that we set in the previous node.

! Important:  
- Be sure to set the below name parameter to exact same topology instance name that we used in the previous node.

In [ ]:
# Activate topology instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceActivate", operationParams)
print(response)

#### 7.5.3.7. De-Activate Topology Instance
In case you dint want to run the topology, at any time you can de-activates it. Below node will de-activate your Media Graph. Which means it will not be running anymore!

! Important:  
- Be sure to set the below name parameter to exact same topology instance name that we used in the previous node.


In [ ]:
# De-Activate topology instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceDeactivate", operationParams)
print(response)

#### 7.5.3.8. Delete Graph Topology Instance
Below code is just a sample to show how to delete previously deployed instance.

In [ ]:
# Delete instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

graphManager.GenericCall("GraphInstanceDelete", operationParams)
print(response)

#### 7.5.3.9. Delete Graph Topology
Below code is just a sample to show how to delete previously deployed toplogy.


! Important:  
- Be sure to set the below name parameter to exact same topology name that you want to delete

In [ ]:
# Delete topology
operationParams = {
                    "name": "SampleTopology"
                    }

graphManager.GenericCall("GraphTopologyDelete", operationParams)
print(response)

## 7.6. Monitoring the output of the LVA edge module

In the sample media graph topology, we defined two "sinks" node:  
Sink 1: sends the inference result (json payload) to IoT Hub
Sink 2: Sends the stream chunks (where motion detected) into Azure Media Services.

### 7.6.1. Monitoring IoT Edge Module to IoT Hub Message flow
In VSCode, under "AZURE IOT HUB" panel (shown in the below screenshot):  
1- Right click on the device name  
2- "Start Monitoring Built-In Event Endpoint"  

<img src="documents/_monitor01.png" width=500px/>   

3- You will see the message flow with inference results and the video chunk logs that are sent to Cloud AMS service  

<img src="documents/_monitor02.png" width=1200px/>   



### 7.6.2. Monitoring the archived video streams in Azure Media Services


On the Azure portal, explore the resource group and open the Azure Media Services resource that we created earlier in this sample. Under the "assets" section, you will see chunks of recording where motion detected.
<img src="documents/_monitor03.png" width=1000px/>   
